In [ ]:
# 특정 종목의 코드를 받으면, 해당 코드의 관련 정보를 확인 후 매수 추천여부를 알려준다.

# 1. 기능
# seeAll = 골든크로스 직전의 종목들 리스트업.
# select = 세부정보 제공, 배당정보 및 차트 제공

# 참고중인 링크

# https://shinminyong.tistory.com/16
# https://m.blog.naver.com/dh3508/222095693865
# https://nakyup.tistory.com/5

# https://md2biz.tistory.com/397
# https://devpouch.tistory.com/47

In [3]:
# 필요한 라이브러리

import FinanceDataReader as web
from datetime import date, timedelta
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import time

import csv
import urllib
from bs4 import BeautifulSoup as bs

%matplotlib inline

In [23]:
# 필요값 (전역변수)

market =""
today = date.today()
startday = date.today() - timedelta(1)
yesterday = date.today() - timedelta(1)
month = 3
outString = ""
date_index = []

In [29]:
# 전체 가동 코드

print("want to decide your settings? it will create data automatically.(y/n)")

while(True):
    menubtn = input()
    print("")
    if menubtn == "y":
        settings()
        time.sleep(3)
        createData(market)
        break
    elif menubtn == "n":
        break
    else:
        print("selected wrong button, want to decide your settings?(y/n)")
        
print("select ur menus.")
print("")
print("seeAll = show all stock datas of market which u selected.")
print("search = search some stocks which you want, it will show charts and recent dividend infos.")

while(True):
    menubtn = input()
    if menubtn == "seeAll":
        analysis()
        break
    elif menubtn == "search":
        print("a")
    else:
        print("you just selected wrong menu.")
        print("")

want to decide your settings? it will create data automatically.(y/n)


 y



market list : 
KRX(한국전체), KOSPI(코스피전체), KOSDAQ(코스닥전체), KONEX(코넥스전체) - 배당 구현중
NASDAQ(나스닥종목), NYSE(뉴욕증권거래소), AMEX(AMEX종목), SP500(S&P500 종목)

'dayago' decieds your startday to crawl data. it needs to set 'dayago' at least 120. (1 year recommanded.)

set your target market.


 NASDAQ


set your 'dayago'.


...

set your investment period. (3/6/9 months only able)


 3



**** Data has been set. make sure ur market. if it doesn't match with the list above, then it will occur some errors. ****

market :  NASDAQ
startday(dayago) :  2020-06-27 ( 360 )
investment period :  3

loading entire stockcode data from market..

list already exists, proceed to next steps.
select ur menus.

seeAll = show all stock datas of market which u selected.
search = search some stocks which you want, it will show charts and recent dividend infos.


 seeAll


loading entire data complete.
loading error occured, restart session and reset your settings.


In [20]:
            SEC = web.DataReader(symbolList[i], startday, yesterday)
            SEC["MA_5"]=SEC["Close"].rolling(window=5).mean()
            SEC["MA_20"]=SEC["Close"].rolling(window=20).mean()
            SEC["MA_60"]=SEC["Close"].rolling(window=60).mean()
            SEC["MA_120"]=SEC["Close"].rolling(window=120).mean() 

            # 데이터 결측치 제거
            SEC.dropna(axis=0)
            
            # 접점을 갖기 위한 포인트 생성
            timing_idx = abs(SEC['MA_20'] - SEC['MA_60']).sort_values().head(10).index
            
            def MA_backtesting(month): #몇개월을 생각하고 주식을 매수하고 매도할 것인지 parameter
                
                if month == 3: #3개월의 날짜를 구한 것
                    delta = date(2018,4,1 ) - date(2018,1,1)
                elif month == 6: #6개월의 날짜를 구한 것
                    delta = date(2018,6,30 ) - date(2018,1,1)
                else:# 9개월의 날짜를 구한 것
                    delta = date(2018,9,28 ) - date(2018,1,1)


                revenue = pd.DataFrame({'date':[], 'revenue':[]})
                
                for i in range(len(date_index)):

                    #3개월/6개월/9개월만큼의 수익률을 구하는 과정에서 만일 3개월/6개월/9개월 뒤의 날짜가 공휴일
                    #이거나 주말이여서 주식시장이 오픈을 하지 않는 것을 대비해서 무조건 산출할 수 있는 예를들어
                    #금요일이면 +3일 - 월요일 / 토요일이면 +2일 - 월요일 /일요일이면 +1일 - 월요일
                    #로 봤을 때 +3일까지의 대비변수를 마련해 놓으면 따로 예외처리하지 않고도 무조건 
                    #결과를 도출할 수 있다.
                    day1 = date(2018,1,2 ) - date(2018,1,1) #예비1일
                    day2 = date(2018,1,3 ) - date(2018,1,1) #예비2일
                    day3 = date(2018,1,4 ) - date(2018,1,1) #예비3일

                    #예외처리로 1일~3일까지 더할 수 있는 코드를 짜서 Error를 피한다.
                    try:
                        a = df.loc[date_index[i] + delta,'Close']
                    except:
                        try:
                            a = df.loc[date_index[i]+ delta+day1,'Close']
                        except:
                            try:
                                a = df.loc[date_index[i]+ delta+day2,'Close']
                            except:
                                a = df.loc[date_index[i]+ delta+day3,'Close']

                    #a = date_index 즉, 접하는 지점으로부터 알고리즘 사용자가 지정한 날을 더한 날짜의 종가를 구한다

                    b = df.loc[date_index[i],'Close'] #date_index의 날짜의 종가데이터를 추출
                    revenue0 = a / b #수익률 및 손실률
                    insert_data =  pd.DataFrame({'date':[date_index[i]], 'revenue':[revenue0]})

                    revenue = revenue.append(insert_data)

                revenue.index = range(len(revenue))

                acc_date0 = revenue.sort_values('revenue',ascending=False)['date'].iloc[0]#최대 수익률의 날짜
                acc_date00 = revenue.sort_values('revenue',ascending=False)['date'].iloc[len(revenue)-1]#최대 손실률의 날짜

                try:
                    a_df = df.loc[acc_date0:acc_date0+delta,'Close']
                except:
                    try:
                        a_df =df.loc[acc_date0:acc_date0+delta+day1,'Close']
                    except:
                        try:
                            a_df = df.loc[acc_date0:acc_date0+delta+day2,'Close']
                        except:
                            a_df = df.loc[acc_date0:acc_date0+delta+day3,'Close']
                #a_df = 최대이익이 날 수 있는 접점날짜부터 알고리즘 사용자가 정한 날까지의 종가 데이터

                try:
                    b_df = df.loc[acc_date00:acc_date00+delta,'Close']
                except:
                    try:
                        b_df =df.loc[acc_date00:acc_date00+delta+day1,'Close']
                    except:
                        try:
                            b_df = df.loc[acc_date00:acc_date00+delta+day2,'Close']
                        except:
                            b_df = df.loc[acc_date00:acc_date00+delta+day3,'Close']

                #b_df = 최대손실이 날 수 있는 접점날짜부터 알고리즘 사용자가 정한 날까지의 종가 데이터         

                plt.subplot(121)
                a_df.plot(figsize=(15,8))#이익률 그래프

                plt.subplot(122)
                b_df.plot(figsize = (15,8),c='b') #손실률 그래프


                acc_date = str(revenue.sort_values('revenue',ascending=False)['date'].iloc[0])   
                acc_revenue = str(revenue.sort_values('revenue',ascending=False)['revenue'].iloc[0])

                acc_date2 = str(revenue.sort_values('revenue',ascending=False)['date'].iloc[len(revenue)-1])
                acc_revenue2 = str(revenue.sort_values('revenue',ascending=False)['revenue'].iloc[len(revenue)-1])

                print('Golden Cross')
                print('이동평균선을 이용하여 고객님에게 맞는 최적의 매수시점은 ' + acc_date + '입니다.')
                print('이동평균선을 이용하여 최적의 매수시점에 주식을 매수하고 '+ str(month) + '개월 뒤에 매도했을 때' + acc_revenue+'배의 수익을 얻을 수 있습니다.\n\n\n' )

                print('Bad Cross')
                print('이동평균선을 이용하여 고객님에게 맞는 최적의 매도시점은 ' + acc_date2 + '입니다.')
                print('이동평균선을 이용하여 최적의 매도시점에 주식을 매도하지않으면 '+ str(month) + '개월 뒤에 매도했을 때' + acc_revenue2+'배의 손실을 얻을 수 있습니다.' )

                return revenue.sort_values('revenue',ascending =False)

In [27]:
# 기본 도표 형성 클래스
def analysis():
    codefilename = './stockDataSet/marketList_' + str(market) + '.csv'

    try:
        with open(codefilename, 'r', newline='') as file:
            print("loading entire data complete.")
            symbolList = codefilename.loc[:, "Symbol"]

            for i in range (0, len(symbolList)):

                # 도표 형성 - 이동평균선 생성 및 매수/매도 포인트 지정
                SEC = web.DataReader(symbolList[i], startday, yesterday)
                SEC["MA_20"]=SEC["Close"].ewm(span=12).mean()
                SEC["MA_60"]=SEC["Close"].ewm(span=26).mean()
                SEC["MACD"] = SEC.apply(lambda x: (x["MA_12"]-x["MA_26"]), axis=1)
                SEC["MACD_signal"] = SEC["MACD"].ewm(span=9).mean()
                SEC["MACD_oscillator"]=SEC.apply(lambda x:(x["MACD"]-x["MACD_signal"]), axis=1)
                
                if (SEC.loc[-1, "MACD"] - )

    except:
        print("loading error occured, restart session and reset your settings.")

In [46]:

codefilename = pd.read_csv("./stockDataSet/marketList_NASDAQ.csv")
symbolList = codefilename.loc[:, "Symbol"]

SEC = web.DataReader(symbolList[0], startday, yesterday).reset_index()
SEC["MA_12"]=SEC["Close"].ewm(span=12).mean()
SEC["MA_26"]=SEC["Close"].ewm(span=26).mean()
SEC["MACD"] = SEC.apply(lambda x: (x["MA_12"]-x["MA_26"]), axis=1)
SEC["MACD_signal"] = SEC["MACD"].ewm(span=9).mean()
SEC["MACD_oscillator"]=SEC.apply(lambda x:(x["MACD"]-x["MACD_signal"]), axis=1)
    
SEC

,Date,Close,Open,High,Low,Volume,Change,MA_12,MA_26,MACD,MACD_signal,MACD_oscillator
0,2020-06-29,90.44,88.31,90.54,87.82,130650000.0,0.0230,90.440000,90.440000,0.000000,0.000000,0.000000
1,2020-06-30,91.20,90.02,91.50,90.00,140220000.0,0.0083,90.851667,90.834615,0.017051,0.009473,0.007578
2,2020-07-01,91.03,91.28,91.84,90.98,110740000.0,-0.0019,90.921270,90.904815,0.016455,0.012334,0.004121
3,2020-07-02,91.03,91.96,92.62,90.91,114040000.0,0.0000,90.955592,90.939811,0.015781,0.013502,0.002279
4,2020-07-06,93.46,92.50,93.94,92.47,118660000.0,0.0268,91.636031,91.524254,0.111777,0.042737,0.069041
...,...,...,...,...,...,...,...,...,...,...,...,...
242,2021-06-15,129.64,130.07,130.59,129.41,62750000.0,-0.0064,127.289806,127.117511,0.172295,-0.458823,0.631119
243,2021-06-16,130.15,130.26,130.89,128.49,91820000.0,0.0039,127.729836,127.342139,0.387696,-0.289519,0.677216
244,2021-06-17,131.79,129.94,132.54,129.66,96720000.0,0.0126,128.354476,127.671611,0.682866,-0.095042,0.777908
245,2021-06-18,130.46,130.71,131.51,130.24,108950000.0,-0.0101,128.678403,127.878158,0.800245,0.084015,0.716230


In [52]:
SEC.loc[0:-1, "MACD"]

Series([], Name: MACD, dtype: float64)

In [ ]:
# seeAll - GC 접근 여부 확인해 줄 로직 설정

In [4]:
# search - 배당정보 형성 클래스 생성
# https://m.blog.naver.com/silvury/221312883764

def getDividendData(stockCode):
    data = []
    url = "https://finance.yahoo.com/quote/" + stockcode + "?p=" + stockcode
    rows = bs(urllib.urlopen(url).read()).findAll('table')[1].tbody.findAll('tr')

In [6]:
# 가동일 기준 데이터를 로드, 스톡리스트를 생성해줄 클래스 생성

# market = url 통해 input값 받아옴
# dayago = 위와 동일

def createData(market):
    
    codefilename = './stockDataSet/marketList_' + str(market) + '.csv'
    
    # 목표 시장 선택 및 데이터 로드
    print("loading entire stockcode data from market..")
    print("")
    try:
        with open(codefilename, 'r', newline='') as file: # file을 여는게 아니라 있는지 확인하는 기능으로 치환
            print("list already exists, proceed to next steps.")
    except:
        try:
            print("")
            print("list none-exists. load marketList which you requested.")
            targetMarket = web.StockListing(market)
            print("loading complete. making lists..")
            print("")

            targetMarket.to_csv('./stockDataSet/marketList_' + str(market) + '.csv')
            print("list created, initialize to next steps.")

        except:
            time.sleep(3)
            print("")
            print("loading error occured, please try again or check ur status.")

In [7]:
# 기본값 입력받을 클래스
def settings():
    
    print("market list : ")
    print("KRX(한국전체), KOSPI(코스피전체), KOSDAQ(코스닥전체), KONEX(코넥스전체) - 배당 구현중")
    print("NASDAQ(나스닥종목), NYSE(뉴욕증권거래소), AMEX(AMEX종목), SP500(S&P500 종목)")
    print("")
    print("'dayago' decieds your startday to crawl data. it needs to set 'dayago' at least 120. (1 year recommanded.)")
    print("")

    
    print("set your target market.")
    global market
    market = input()
    
    print("set your 'dayago'.")
    dayago = input()
    
    print("set your investment period. (3/6/9 months only able)")
    global month
    month = input()

    global startday
    startday = date.today() - timedelta(int(dayago))
        
    print("")
    print("**** Data has been set. make sure ur market. if it doesn't match with the list above, then it will occur some errors. ****")
    print("")

    print("market : ", market)
    print("startday(dayago) : ", startday, "(", dayago, ")")
    print("investment period : ", month)
    print("")